## Read dataframe

In [1]:
import numpy as np
import pandas as pd

DATASET_PATH = "./LUMEN_DS.csv"
ENCODING = "UTF-16"
SEPARATOR = "|"
NA_VALUES = "NaN"
ROWS_LIMIT = None

manufacturing_region = "manufacturing_region"
manufacturing_location_code = "manufacturing_location_code"
intercompany = "intercompany"
customer_id = "customer_id"
customer_industry = "customer_industry"
customer_region = "customer_region"
customer_first_invoice_date = "customer_first_invoice_date"
top_customer_group = "top_customer_group"
item_code = "item_code"
product_family = "product_family"
product_group = "product_group"
price_last_modified_date_in_the_erp = "price_last_modified_date_in_the_erp"
born_on_date = "born_on_date"
make_vs_buy = "make_vs_buy"
sales_channel_internal = "sales_channel_internal"
sales_channel_external = "sales_channel_external"
sales_channel_grouping = "sales_channel_grouping"
invoice_date = "invoice_date"
invoice_num = "invoice_num"
invoice_line_num = "invoice_line_num"
order_date = "order_date"
order_num = "order_num"
order_line_num = "order_line_num"
invoiced_qty_shipped = "invoiced_qty_shipped"
ordered_qty = "ordered_qty"
invoiced_price = "invoiced_price"
invoiced_price_tx = "invoiced_price_tx"
cost_of_part = "cost_of_part"
material_cost_of_part = "material_cost_of_part"
labor_cost_of_part = "labor_cost_of_part"
overhead_cost_of_part = "overhead_cost_of_part"
gm = "gm"
num_of_unique_products_on_a_quote = "num_of_unique_products_on_a_quote"

STRING_COLUMNS = [
    manufacturing_region,
    manufacturing_location_code,
    intercompany,
    customer_id,
    customer_industry,
    customer_region,
    top_customer_group,
    item_code,
    product_family,
    product_group,
    make_vs_buy,
    sales_channel_internal,
    sales_channel_external,
    sales_channel_grouping,
    invoice_num,
    invoice_line_num,
    order_num,
    order_line_num,
]

DATE_ONLY_COLUMNS = [born_on_date, invoice_date, order_date]

DATETIME_COLUMNS = [price_last_modified_date_in_the_erp, customer_first_invoice_date]

INT_COLUMNS = [num_of_unique_products_on_a_quote]

FLOAT_COLUMNS = [
    invoiced_qty_shipped,
    ordered_qty,
    invoiced_price,
    invoiced_price_tx,
    cost_of_part,
    material_cost_of_part,
    labor_cost_of_part,
    overhead_cost_of_part,
    gm,
]

DATE_COLUMNS = DATE_ONLY_COLUMNS + DATETIME_COLUMNS
NUMERIC_COLUMNS = INT_COLUMNS + FLOAT_COLUMNS
ALL_COLUMNS = STRING_COLUMNS + DATE_COLUMNS + NUMERIC_COLUMNS

df = pd.read_csv(DATASET_PATH, sep=SEPARATOR, encoding=ENCODING, nrows=ROWS_LIMIT)

## Fix columns

In [2]:
for col in STRING_COLUMNS:
    df[col] = df[col].astype("str")
    df[col] = df[col].replace("nan", np.nan, regex=True)

for col in DATE_COLUMNS:
    df[col] = pd.to_datetime(df[col], errors="coerce")

## Impute cost_of_part with past or future values

In [3]:
"""
indices = df.index
problem_indices = indices[df["cost_of_part"] == 0]
problem_indices = problem_indices.tolist()

for index in problem_indices:
    problem_value = df.loc[index, :]
    problem_item_code = problem_value["item_code"]
    last_value_cost_of_part = df[
        (df["item_code"] == problem_item_code)
        & (df["invoice_date"] <= problem_value["invoice_date"])
        & (df["cost_of_part"] != 0)
    ]
    if len(last_value_cost_of_part) > 0:
        max_invoice_date = last_value_cost_of_part["invoice_date"].max()
        last_cost = (
            last_value_cost_of_part.groupby(by="invoice_date").mean().reset_index()
        )
        df.loc[index, ["cost_of_part"]] = last_cost["cost_of_part"][
            last_cost["invoice_date"] == max_invoice_date
        ].values[0]

df["gm"] = (df["invoiced_price"] - df["cost_of_part"]) / df["invoiced_price"]
"""

'\nindices = df.index\nproblem_indices = indices[df["cost_of_part"] == 0]\nproblem_indices = problem_indices.tolist()\n\nfor index in problem_indices:\n    problem_value = df.loc[index, :]\n    problem_item_code = problem_value["item_code"]\n    last_value_cost_of_part = df[\n        (df["item_code"] == problem_item_code)\n        & (df["invoice_date"] <= problem_value["invoice_date"])\n        & (df["cost_of_part"] != 0)\n    ]\n    if len(last_value_cost_of_part) > 0:\n        max_invoice_date = last_value_cost_of_part["invoice_date"].max()\n        last_cost = (\n            last_value_cost_of_part.groupby(by="invoice_date").mean().reset_index()\n        )\n        df.loc[index, ["cost_of_part"]] = last_cost["cost_of_part"][\n            last_cost["invoice_date"] == max_invoice_date\n        ].values[0]\n\ndf["gm"] = (df["invoiced_price"] - df["cost_of_part"]) / df["invoiced_price"]\n'

In [4]:
df[df.cost_of_part == 0]

,manufacturing_region,manufacturing_location_code,intercompany,customer_id,customer_industry,customer_region,customer_first_invoice_date,top_customer_group,item_code,product_family,...,invoiced_qty_shipped,ordered_qty,invoiced_price,invoiced_price_tx,cost_of_part,material_cost_of_part,labor_cost_of_part,overhead_cost_of_part,gm,num_of_unique_products_on_a_quote
2,Asia,N13,YES,235861,IC000,North America,2008-02-07,OTHER,057280,PF002,...,3.0,3.0,0.00,0.00,0.0,0.0,0.0,0.0,NaN,14.0
3,Asia,N13,NO,223839,IC000,Asia,2003-09-08,OTHER,309146,PF002,...,2000.0,5000.0,0.20,0.20,0.0,0.0,0.0,0.0,1.0,1.0
4,Asia,N13,NO,223839,IC000,Asia,2003-09-08,OTHER,309146,PF002,...,1230.0,5000.0,0.20,0.20,0.0,0.0,0.0,0.0,1.0,1.0
5,Asia,N13,NO,223839,IC000,Asia,2003-09-08,OTHER,309146,PF002,...,1770.0,5000.0,0.20,0.20,0.0,0.0,0.0,0.0,1.0,1.0
6,Asia,N13,NO,224307,IC000,Asia,2011-05-27,OTHER,343372,PF002,...,4000.0,4000.0,2.78,2.78,0.0,0.0,0.0,0.0,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294898,Asia,N13,NO,224600,IC007,Asia,2012-04-10,STAR,654732,PF002,...,81.0,81.0,12.00,12.00,0.0,0.0,0.0,0.0,1.0,1.0
1294899,Asia,N13,NO,231794,IC000,Asia,2011-06-14,OTHER,658627,PF002,...,1050.0,1050.0,2.12,2.12,0.0,0.0,0.0,0.0,1.0,2.0
1294900,North America,B21,NO,106730,IC009,North America,2016-09-08,OTHER,SHIPPING CHARGES,PF001,...,1.0,1.0,54.23,54.23,0.0,0.0,0.0,0.0,1.0,1.0
1294901,Asia,B70,YES,19245,IC000,Asia,2005-03-08,OTHER,JAB-07-09750,PF001,...,5.0,5.0,0.00,0.00,0.0,0.0,0.0,0.0,NaN,1.0


## Try to impute missing manufacturing_region 

In [5]:
df_manufacturing = df[
    ~df.manufacturing_region.isna() & ~df.manufacturing_location_code.isna()
]

manufacturing_dict = dict(
    zip(
        df_manufacturing.manufacturing_location_code,
        df_manufacturing.manufacturing_region,
    )
)

In [6]:
df[manufacturing_region] = df.apply(
    lambda x: manufacturing_dict[x[manufacturing_location_code]]
    if x[manufacturing_location_code] in manufacturing_dict.keys()
    else x[manufacturing_region],
    axis=1,
)

## Filter df on conditions

In [7]:
preconditions = (
    (df.cost_of_part > 0)
    & (df.invoiced_price > 0)
    & (df.gm < 1)
    & (df.gm > 0)
    & (df.ordered_qty > 0)
    & (df.invoiced_qty_shipped > 0)
    & (df.intercompany == "NO")
    & (df.make_vs_buy != "RAW MATERIAL")
    & (df.make_vs_buy != "BUY - CUST. SUPPLIED")
    & (df.make_vs_buy != "BUY - INTERPLNT TRNS")
    & (df.make_vs_buy != "PURCHASED (RAW)")
    & (df.customer_id) > 0
)

df = df[preconditions].reset_index(drop=True)

## Relabel columns

### Make vs buy

In [8]:
make_cols = ["MANUFACTURED", "RAW MATERIAL", "FINISHED GOODS"]
buy_cols = [
    "BUY",
    "BUY - IMPORTED",
    "BUY - LOCAL",
    "BUY - CUST. SUPPLIED",
    "BUY - INTERPLNT TRNS",
    "PURCHASED",
    "PURCHASED (RAW)",
]


def relabel_make_vs_buy(x):
    if x in make_cols:
        return "MAKE"
    elif x in buy_cols:
        return "BUY"
    else:
        return x


df[make_vs_buy] = df.make_vs_buy.apply(lambda x: relabel_make_vs_buy(x))

### STAR customer

In [9]:
def relabel_customer_region(region, group):
    if group == "STAR":
        return "STAR"
    else:
        return region


df[customer_region] = df.apply(
    lambda x: relabel_customer_region(x[customer_region], x[top_customer_group]), axis=1
)

## Add new features

In [10]:
new_old_customer = "new_old_customer"

df[new_old_customer] = df.customer_first_invoice_date.apply(lambda x: "NEW" if x.year >= 2015 else "OLD")

## Arrange ordered_qty into buckets 

In [11]:
# TODO

## Remove columns

In [12]:
cols_to_remove = [
    customer_id,
    manufacturing_location_code,
    customer_first_invoice_date,
    price_last_modified_date_in_the_erp,
    born_on_date,
    invoice_date,
    order_date,
    invoiced_price_tx,
    invoiced_price,
    cost_of_part,
    sales_channel_internal,
    sales_channel_external,
    sales_channel_grouping,
    top_customer_group,
    invoice_num,
    invoice_line_num,
    order_num,
    order_line_num,
    material_cost_of_part,
    labor_cost_of_part,
    overhead_cost_of_part,
    item_code,
    intercompany,
    product_group,
    invoiced_qty_shipped,
    num_of_unique_products_on_a_quote,
]

df = df.drop(cols_to_remove, axis=1)

In [13]:
df

,manufacturing_region,customer_industry,customer_region,product_family,make_vs_buy,ordered_qty,gm,new_old_customer
0,North America,IC012,North America,PF002,BUY,1000.0,0.816429,OLD
1,Asia,IC009,Asia,PF002,MAKE,21.0,0.526436,NEW
2,Asia,IC009,Asia,PF002,MAKE,21.0,0.521512,NEW
3,Asia,IC008,Asia,PF002,MAKE,1550.0,0.409060,OLD
4,Asia,IC008,Asia,PF002,MAKE,1550.0,0.332513,NEW
...,...,...,...,...,...,...,...,...
685233,North America,IC009,North America,PF002,BUY,24.0,0.426160,NEW
685234,North America,IC009,North America,PF002,BUY,32.0,0.380507,NEW
685235,North America,IC008,North America,PF002,MAKE,15.0,0.892857,OLD
685236,North America,IC000,Asia,PF002,BUY,10000.0,0.358974,OLD
